# Interactive Chat Bot

## Objective
Build and fine-tune a system that leverages pre-trained transformer models to address real-world tasks. This system will help a student seeking an internship by performing various tasks involving natrual language understanding and generation.

## Specifications
1. Interactive Command Line Interface
    * Create a text-based program that interacts with users through open-ended plain-text prompts.
3. Sentence Auto-Completion
    * Implement functionality that predicts and completes common sentences based on context. Use your transformer model to generate plausible continuations for user-provided text.
5. Cover Letter Analysis
    * Design a feature where the system reads a provided cover letter and extracts meaningful information about the applicant.
7. Dynamic Cover Letter Generation
    * Enable the system to generate a personalized cover letter based on user-specified constraints, such as desired role, key skills, or industry. Combine a flexible template with dynamic content generation to meet specific user needs.

In [ ]:
import pandas as pd
import torch

menu_df = pd.read_csv("https://docs.google.com/spreadsheets/d/1xl_orOAWc--j_p4MOCkgrbjg2feqAsoA4MZ6qNYOYic/export?format=csv")

menu_df.tail()

,Text,Label
298,“Exit confirmed. Thank you for using our progr...,Exit
299,“Thank you for visiting. See you next time!” P...,Exit
300,“Logging you out securely…” Professional close.,Exit
301,“Saving changes. Application shutting down.” S...,Exit
302,“Goodbye! Closing application.” Warm and invit...,Exit


In [ ]:
menu_df.to_csv('input.csv',index=False)

In [ ]:
!pip install datasets

from datasets import load_dataset

dataset = load_dataset("csv",data_files={"train":"input.csv"},split='train')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['Text', 'Label'],
    num_rows: 303
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/303 [00:00<?, ? examples/s]

In [ ]:
from sklearn.preprocessing import LabelBinarizer

enc = LabelBinarizer()
enc.fit(menu_df['Label'])

LabelBinarizer()

In [ ]:
enc.transform(menu_df['Label'])[:10]

array([[0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [ ]:
import torch
def add_labels(example):
    example['labels'] = torch.tensor(enc.transform([example['Label']])[0]*1.0)
    return example

tokenized_datasets = tokenized_datasets.map(add_labels)

Map:   0%|          | 0/303 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

Dataset({
    features: ['Text', 'Label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 303
})

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["Text"])
tokenized_datasets = tokenized_datasets.remove_columns(["Label"])
#tokenized_datasets = tokenized_datasets.rename_column("Label", "labels")
tokenized_datasets.set_format("torch")

In [ ]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 303
})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

model_ckpt = "bert-base-uncased"  # etc.
num_labels = 3  # etc.
# 1. Example: Help me write a cover letter
# 2. Example: Help me read a cover letter
# 3. Example: Exit

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=num_labels,
    problem_type="multi_label_classification",  # this is important
)

# Tokenize input text
text = "This is a great example."
inputs = tokenizer(text, return_tensors="pt")

# Get model output
outputs = model(**inputs)

# Process output
logits = outputs.logits
predictions = torch.argmax(logits, dim=1)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
logits.shape

torch.Size([1, 3])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [ ]:
# here is a loop just showing how we are going to do this

# Create a DataLoader
train_dataloader = DataLoader(tokenized_datasets, batch_size=5, shuffle=True)

# Iterate through the DataLoader
for batch in train_dataloader:
    break

In [ ]:
batch

{'input_ids': tensor([[ 101, 2079, 2017,  ...,    0,    0,    0],
         [ 101, 1045, 1521,  ...,    0,    0,    0],
         [ 101, 2043, 1996,  ...,    0,    0,    0],
         [ 101, 2071, 2017,  ...,    0,    0,    0],
         [ 101, 2054, 1521,  ...,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[0., 1., 0.],
         [0., 0., 1.],
         [1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]])}

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/183 [00:00<?, ?it/s]

In [ ]:
text = "Stop this darn program."
inputs = tokenizer(text, return_tensors="pt").to(device)

# Get model output
outputs = model(**inputs)

# Process output
logits = outputs.logits
predictions = torch.argmax(logits, dim=1)

In [ ]:
logits,predictions,enc.inverse_transform(logits.cpu().detach().numpy())

(tensor([[ 4.2353, -3.5931, -3.5777]], device='cuda:0',
        grad_fn=<AddmmBackward0>),
 tensor([0], device='cuda:0'),
 array(['Exit'], dtype='<U5'))

In [ ]:
text = "Please help me write the best cover letter to get a deep learning internship"
inputs = tokenizer(text, return_tensors="pt").to(device)

# Get model output
outputs = model(**inputs)

# Process output
logits = outputs.logits
predictions = torch.argmax(logits, dim=1)

In [ ]:
logits,predictions,enc.inverse_transform(logits.cpu().detach().numpy())

In [ ]:
text = "I am a recruiting manager and I need help reading a resume"
inputs = tokenizer(text, return_tensors="pt").to(device)

# Get model output
outputs = model(**inputs)

# Process output
logits = outputs.logits
predictions = torch.argmax(logits, dim=1)

In [ ]:
logits,predictions,enc.inverse_transform(logits.cpu().detach().numpy())

(tensor([[-3.2274, -3.1235,  2.7861]], device='cuda:0',
        grad_fn=<AddmmBackward0>),
 tensor([2], device='cuda:0'),
 array(['Write'], dtype='<U5'))

In [ ]:
import torch

# Save the model's state dictionary
torch.save(model.state_dict(), 'my_model.pth')

# Save the tokenizer (optional but recommended)
tokenizer.save_pretrained('my_tokenizer')

# To load the model later:
# model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)
# model.load_state_dict(torch.load('my_model.pth'))
# tokenizer = AutoTokenizer.from_pretrained('my_tokenizer')

('my_tokenizer/tokenizer_config.json',
 'my_tokenizer/special_tokens_map.json',
 'my_tokenizer/vocab.txt',
 'my_tokenizer/added_tokens.json',
 'my_tokenizer/tokenizer.json')

In [28]:
# load model, test a user input
import torch
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)
model.load_state_dict(torch.load('my_model.pth', map_location=torch.device('cpu')))
tokenizer = AutoTokenizer.from_pretrained('my_tokenizer')

text = input()
inputs = tokenizer(text, return_tensors="pt")

# Get model output
outputs = model(**inputs)

# Process output
logits = outputs.logits
predictions = torch.argmax(logits, dim=1)
logits,predictions,enc.inverse_transform(logits.cpu().detach().numpy())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/2l/t4s4q4z574bgmnvp9qx2z_n00000gn/T/ipykernel_85040/1875054278.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via 

 help me read this 


(tensor([[-2.9808,  2.7593, -3.2443]], grad_fn=<AddmmBackward0>),
 tensor([1]),
 array(['Read'], dtype='<U5'))

fatal: not a git repository (or any of the parent directories): .git
